# Parallel Sub-Agent Research

This notebook demonstrates **parallel sub-agent delegation** - spawning multiple agents to research sub-questions simultaneously.

| Pattern | Notebook 01 | This Notebook |
|---------|:-----------:|:-------------:|
| Sequential searches | ✅ | ❌ |
| Parallel sub-agents | ❌ | ✅ |
| Isolated context per question | ❌ | ✅ |
| Speed | Slower | **Faster** |

**Architecture:**
```
                    ┌─────────────┐
                    │   Planner   │
                    │   Agent     │
                    └──────┬──────┘
                           │ Creates sub-questions
           ┌───────────────┼───────────────┐
           ▼               ▼               ▼
    ┌────────────┐  ┌────────────┐  ┌────────────┐
    │ Sub-Agent  │  │ Sub-Agent  │  │ Sub-Agent  │
    │     #1     │  │     #2     │  │     #3     │
    │ (Question) │  │ (Question) │  │ (Question) │
    └─────┬──────┘  └─────┬──────┘  └─────┬──────┘
          │               │               │
          └───────────────┼───────────────┘
                          ▼
                   ┌─────────────┐
                   │ Synthesizer │
                   │   Agent     │
                   └─────────────┘
```

## Step 1: Setup

In [1]:
#!pip install openhands-sdk litellm python-dotenv tavily-python

In [2]:
import os
import json
import asyncio
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv
load_dotenv()

from typing import List
from pydantic import Field
from tavily import TavilyClient
from openhands.sdk import LLM, Agent, Conversation, Tool, Action, Observation, ToolDefinition, TextContent
from openhands.sdk.tool import register_tool, ToolExecutor
from openhands.tools.file_editor import FileEditorTool

print(f"Model: {os.getenv('LLM_MODEL', 'openai/gpt-4o')}")

# Observability status
if os.getenv("LMNR_PROJECT_API_KEY"):
    print("✓ Observability: Laminar tracing enabled")
else:
    print("ℹ Observability: Set LMNR_PROJECT_API_KEY for tracing")

<frozen abc>:106: DeprecationWarning: You should use `Logger` instead. Deprecated since version 1.39.0 and will be removed in a future release.
<frozen abc>:106: DeprecationWarning: You should use `LoggerProvider` instead. Deprecated since version 1.39.0 and will be removed in a future release.
/Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/opentelemetry/_events/__init__.py:201: DeprecationWarning: You should use `ProxyLoggerProvider` instead. Deprecated since version 1.39.0 and will be removed in a future release.
  _PROXY_EVENT_LOGGER_PROVIDER = ProxyEventLoggerProvider()
2026-01-23 21:11:05,082::lmnr.opentelemetry_lib.litellm::WARNING: Laminar LiteLLM callback is deprecated. LiteLLM is already instrumented by Laminar. This callback will not have any effect. You can safely remove the callback from your code. (__init__.py:83)


Model: openai/gpt-5
✓ Observability: Laminar tracing enabled


## Step 2: Create LLM and Tavily Tool

In [3]:
# Create LLMs
llm = LLM(
    model="openai/gpt-4o",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

synthesis_llm = LLM(
    model="openai/gpt-5.1",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

# Tavily client and tool
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

class SearchAction(Action):
    query: str = Field(description="Search query")

class SearchObservation(Observation):
    results: str = Field(description="Results")
    @property
    def to_llm_content(self): return [TextContent(text=self.results)]

class SearchExecutor(ToolExecutor):
    def __call__(self, action: SearchAction, conversation=None) -> SearchObservation:
        try:
            r = tavily.search(query=action.query, max_results=5)
            text = "\n\n".join([
                f"**{x['title']}**\n{x['content'][:300]}\nSource: {x['url']}"
                for x in r['results']
            ])
            return SearchObservation(results=text or "No results found")
        except Exception as e:
            return SearchObservation(results=f"Search failed: {str(e)}")

class SearchTool(ToolDefinition[SearchAction, SearchObservation]):
    @classmethod
    def create(cls, conv_state) -> List["SearchTool"]:
        return [cls(description="Web search via Tavily",
                    action_type=SearchAction, observation_type=SearchObservation, 
                    executor=SearchExecutor())]

register_tool("TavilySearch", SearchTool.create)
print("✓ LLMs and Tavily tool ready")

✓ LLMs and Tavily tool ready


## Step 3: Define Sub-Agent Research Function

Each sub-agent researches ONE sub-question independently.

In [4]:
cwd = os.getcwd()

def research_sub_question(question: str, index: int) -> dict:
    """
    Spawn a sub-agent to research a single sub-question.
    Each agent writes its findings to a unique file.
    """
    print(f"  🔍 Sub-agent {index+1} starting: {question[:50]}...")
    
    # Create a dedicated agent for this sub-question
    sub_agent = Agent(
        llm=llm,
        tools=[Tool(name="TavilySearch"), Tool(name=FileEditorTool.name)],
    )
    
    # Create isolated conversation for this sub-agent
    sub_conversation = Conversation(agent=sub_agent, workspace=cwd)
    
    # Each sub-agent writes to its own file
    output_file = f"findings_{index+1}.md"
    
    prompt = f"""
    Research this specific question: {question}
    
    1. Use TavilySearch to find 2-3 relevant sources
    2. Write your findings to the file `{output_file}` including:
       - Key points discovered
       - Source URLs for citations
    
    Be concise but thorough.
    """
    
    sub_conversation.send_message(prompt)
    sub_conversation.run()
    
    # Read findings from the file the agent created
    findings = ""
    try:
        with open(output_file, "r") as f:
            findings = f.read()
    except FileNotFoundError:
        findings = f"[Sub-agent {index+1} did not write findings]"
    
    print(f"  ✓ Sub-agent {index+1} complete")
    
    return {
        "question": question,
        "findings": findings,
        "index": index
    }

print("✓ Sub-agent research function ready")

✓ Sub-agent research function ready


## Step 4: Run Parallel Research

Uses ThreadPoolExecutor to run sub-agents in parallel.

In [5]:
import time

# Research topic and sub-questions
topic = "Latest breakthroughs in AI agents and autonomous systems (2024-2025)"

sub_questions = [
    "What are the major advances in LLM-based autonomous agents in 2024-2025?",
    "How has multi-agent collaboration evolved in AI systems?",
    "What new tools and frameworks have emerged for building AI agents?",
    "What are the key challenges and limitations of current AI agents?",
    "What real-world applications are using AI agents successfully?",
]

print("=" * 60)
print(f"PARALLEL RESEARCH: {topic}")
print("=" * 60)
print(f"\nResearching {len(sub_questions)} sub-questions in parallel...\n")

# Run sub-agents in parallel using ThreadPoolExecutor
start_time = time.time()

with ThreadPoolExecutor(max_workers=len(sub_questions)) as executor:
    # Submit all sub-agent tasks
    futures = [
        executor.submit(research_sub_question, q, i) 
        for i, q in enumerate(sub_questions)
    ]
    
    # Collect results as they complete
    results = [f.result() for f in futures]

elapsed = time.time() - start_time
print(f"\n✓ All {len(sub_questions)} sub-agents complete in {elapsed:.1f}s")

# Sort by original index
results.sort(key=lambda x: x['index'])

# Save raw findings to file
findings_text = f"# Research Findings: {topic}\n\n"
for r in results:
    findings_text += f"## {r['index']+1}. {r['question']}\n\n"
    findings_text += f"{r['findings']}\n\n---\n\n"

with open("parallel_findings.md", "w") as f:
    f.write(findings_text)

print("✓ Raw findings saved to parallel_findings.md")

PARALLEL RESEARCH: Latest breakthroughs in AI agents and autonomous systems (2024-2025)

Researching 5 sub-questions in parallel...

  🔍 Sub-agent 1 starting: What are the major advances in LLM-based autonomou...
  🔍 Sub-agent 2 starting: How has multi-agent collaboration evolved in AI sy...


[01/23/26 21:11:05] INFO     Created new conversation 727dfcc5-ddf8-43f6-a30f-7c3a70002240             ]8;id=971573;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py\state.py]8;;\:]8;id=625249;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py#302\302]8;;\
                             State: {'id': UUID('727dfcc5-ddf8-43f6-a30f-7c3a70002240'), 'workspace':              
                             {'working_dir': '/Users/rajivshah/Code/openhands-deep-agent', 'kind':                 
                             'LocalWorkspace'}, 'persistence_dir': None, 'max_iterations': 500,                    
                             'stuck_detection': True, 'execution_status':                                          
                             <ConversationExecutionStatus.IDLE: 'idle'>, 'confirmation_policy':                    
                             {'kind': 'NeverConfirm'}, 'security_analyzer': None,                                  
                             'activated_knowledge_skills': [], 'blocked_actions': {},                              
                             'blocked_messages': {}, 'stats': {'usage_to_metrics': {}},                            
                             'secret_registry': {'secret_sources': {}}}                                            
                             Agent: {'llm': {'model': 'openai/gpt-4o', 'api_key':                                  
                             SecretStr('**********'), 'openrouter_site_url':                                       
                             'https://docs.all-hands.dev/', 'openrouter_app_name': 'OpenHands',                    
                             'num_retries': 5, 'retry_multiplier': 8.0, 'retry_min_wait': 8,                       
                             'retry_max_wait': 64, 'timeout': 300, 'max_message_chars': 30000,                     
                             'temperature': 0.0, 'top_p': 1.0, 'max_input_tokens': 128000,                         
                             'max_output_tokens': 16384, 'stream': False, 'drop_params': True,                     
                             'modify_params': True, 'disable_stop_word': False, 'caching_prompt':                  
                             True, 'log_completions': False, 'log_completions_folder':                             
                             'logs/completions', 'native_tool_calling': True, 'reasoning_effort':                  
                             'high', 'enable_encrypted_reasoning': True, 'prompt_cache_retention':                 
                             '24h', 'extended_thinking_budget': 200000, 'usage_id': 'default',                     
                             'litellm_extra_body': {}}, 'tools': [{'name': 'TavilySearch', 'params':               
                             {}}, {'name': 'file_editor', 'params': {}}], 'mcp_config': {},                        
                             'include_default_tools': ['FinishTool', 'ThinkTool'],                                 
                             'system_prompt_filename': 'system_prompt.j2', 'security_policy_filename':             
                             'security_policy.j2', 'system_prompt_kwargs': {'llm_security_analyzer':               
                             True}, 'kind': 'Agent'}                                                               

  🔍 Sub-agent 3 starting: What new tools and frameworks have emerged for bui...
  🔍 Sub-agent 4 starting: What are the key challenges and limitations of cur...
  🔍 Sub-agent 5 starting: What real-world applications are using AI agents s...


[01/23/26 21:11:05] INFO     Created new conversation d1eb1866-4114-4cfc-b351-7b5f203640a3             ]8;id=285751;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py\state.py]8;;\:]8;id=889413;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py#302\302]8;;\
                             State: {'id': UUID('d1eb1866-4114-4cfc-b351-7b5f203640a3'), 'workspace':              
                             {'working_dir': '/Users/rajivshah/Code/openhands-deep-agent', 'kind':                 
                             'LocalWorkspace'}, 'persistence_dir': None, 'max_iterations': 500,                    
                             'stuck_detection': True, 'execution_status':                                          
                             <ConversationExecutionStatus.IDLE: 'idle'>, 'confirmation_policy':                    
                             {'kind': 'NeverConfirm'}, 'security_analyzer': None,                                  
                             'activated_knowledge_skills': [], 'blocked_actions': {},                              
                             'blocked_messages': {}, 'stats': {'usage_to_metrics': {}},                            
                             'secret_registry': {'secret_sources': {}}}                                            
                             Agent: {'llm': {'model': 'openai/gpt-4o', 'api_key':                                  
                             SecretStr('**********'), 'openrouter_site_url':                                       
                             'https://docs.all-hands.dev/', 'openrouter_app_name': 'OpenHands',                    
                             'num_retries': 5, 'retry_multiplier': 8.0, 'retry_min_wait': 8,                       
                             'retry_max_wait': 64, 'timeout': 300, 'max_message_chars': 30000,                     
                             'temperature': 0.0, 'top_p': 1.0, 'max_input_tokens': 128000,                         
                             'max_output_tokens': 16384, 'stream': False, 'drop_params': True,                     
                             'modify_params': True, 'disable_stop_word': False, 'caching_prompt':                  
                             True, 'log_completions': False, 'log_completions_folder':                             
                             'logs/completions', 'native_tool_calling': True, 'reasoning_effort':                  
                             'high', 'enable_encrypted_reasoning': True, 'prompt_cache_retention':                 
                             '24h', 'extended_thinking_budget': 200000, 'usage_id': 'default',                     
                             'litellm_extra_body': {}}, 'tools': [{'name': 'TavilySearch', 'params':               
                             {}}, {'name': 'file_editor', 'params': {}}], 'mcp_config': {},                        
                             'include_default_tools': ['FinishTool', 'ThinkTool'],                                 
                             'system_prompt_filename': 'system_prompt.j2', 'security_policy_filename':             
                             'security_policy.j2', 'system_prompt_kwargs': {'llm_security_analyzer':               
                             True}, 'kind': 'Agent'}                                                               

[01/23/26 21:11:05] INFO     Created new conversation ecc94598-4d7d-4023-bb1e-7e8db3809f16             ]8;id=833737;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py\state.py]8;;\:]8;id=266031;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py#302\302]8;;\
                             State: {'id': UUID('ecc94598-4d7d-4023-bb1e-7e8db3809f16'), 'workspace':              
                             {'working_dir': '/Users/rajivshah/Code/openhands-deep-agent', 'kind':                 
                             'LocalWorkspace'}, 'persistence_dir': None, 'max_iterations': 500,                    
                             'stuck_detection': True, 'execution_status':                                          
                             <ConversationExecutionStatus.IDLE: 'idle'>, 'confirmation_policy':                    
                             {'kind': 'NeverConfirm'}, 'security_analyzer': None,                                  
                             'activated_knowledge_skills': [], 'blocked_actions': {},                              
                             'blocked_messages': {}, 'stats': {'usage_to_metrics': {}},                            
                             'secret_registry': {'secret_sources': {}}}                                            
                             Agent: {'llm': {'model': 'openai/gpt-4o', 'api_key':                                  
                             SecretStr('**********'), 'openrouter_site_url':                                       
                             'https://docs.all-hands.dev/', 'openrouter_app_name': 'OpenHands',                    
                             'num_retries': 5, 'retry_multiplier': 8.0, 'retry_min_wait': 8,                       
                             'retry_max_wait': 64, 'timeout': 300, 'max_message_chars': 30000,                     
                             'temperature': 0.0, 'top_p': 1.0, 'max_input_tokens': 128000,                         
                             'max_output_tokens': 16384, 'stream': False, 'drop_params': True,                     
                             'modify_params': True, 'disable_stop_word': False, 'caching_prompt':                  
                             True, 'log_completions': False, 'log_completions_folder':                             
                             'logs/completions', 'native_tool_calling': True, 'reasoning_effort':                  
                             'high', 'enable_encrypted_reasoning': True, 'prompt_cache_retention':                 
                             '24h', 'extended_thinking_budget': 200000, 'usage_id': 'default',                     
                             'litellm_extra_body': {}}, 'tools': [{'name': 'TavilySearch', 'params':               
                             {}}, {'name': 'file_editor', 'params': {}}], 'mcp_config': {},                        
                             'include_default_tools': ['FinishTool', 'ThinkTool'],                                 
                             'system_prompt_filename': 'system_prompt.j2', 'security_policy_filename':             
                             'security_policy.j2', 'system_prompt_kwargs': {'llm_security_analyzer':               
                             True}, 'kind': 'Agent'}                                                               

...

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: file_editor
Result:
[File /Users/rajivshah/Code/openhands-deep-agent/findings_3.md edited with 1 changes.]
[begin of edit 1 / 1]
(content before edit)
 1|
(content after edit)
+1|# Findings on New Tools and Frameworks for Building AI Agents
 2|
+3|## Key Points Discovered
+4|
+5|1. **Google Agent Dev Kit (ADK)**
+6|   - Integrates natively with Gemini and Vertex AI.
+7|   - Features a modular, component-based architecture.
+8|   - Supports hierarchical agent compositions.
+9|   - Allows efficient development with under 100 lines of code.
+10|   - Supports custom tool development.
+11|   - [Source](https://www.datacamp.com/blog/best-ai-agents)
+12|
+13|2. **LangChain**
+14|   - Connects large language models (LLMs) with external data, APIs, and AI agent tools.
+15|   - Ideal for task automation and multi-step workflows.
+16|   - [Source](https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents)
+17|
+18|3. **OpenAI Responses API**
+19|   - Combines the simplicity of the Chat Completions API with the tool use capabilities of the Assistants API.
+20|   - Serves as a new API primitive for leveraging OpenAI’s built-in tools to build agents.
+21|   - [Source](https://openai.com/index/new-tools-for-building-agents/)
+22|
+23|## Source URLs for Citations
+24|- https://www.datacamp.com/blog/best-ai-agents
+25|- https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents
+26|- https://openai.com/index/new-tools-for-building-agents/
[end of edit 1 / 1]

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Summary: Documented LLM advances 2024-2025 in findings_1.md with sources.

Finish with message:
I have researched the major advances in LLM-based autonomous agents for 2024-2025 and documented the findings in 
the file `findings_1.md`. Here are the key points:

1. **Domain-specific LLMs**: There is a trend towards developing domain-specific LLMs tailored to specific 
industries, enhancing efficiency and effectiveness.
   - Source: [Medium](https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf)

2. **Era of Autonomy**: The period of 2025-2026 is marked as the "Era of Autonomy," with a shift towards 
sophisticated autonomous agents like Manus AI.
   - Source: [Capitole 
Consulting](https://www.capitole-consulting.com/blog/turing-to-autonomous-agents-2025-llm-ecosystem)

3. **Multi-agent LLMs**: Multi-agent systems, where LLM-driven agents collaborate to solve complex tasks, are 
becoming more prevalent.
   - Source: [SuperAnnotate](https://www.superannotate.com/blog/multi-agent-llms)

The file includes these key points along with the source URLs for citations.

Tokens: ↑ input 60.28K • cache hit 73.47% • ↓ output 3.41K • $ 0.1295

  ✓ Sub-agent 1 complete


Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Summary: Documented new AI agent tools in findings_3.md with sources.

Finish with message:
I have researched the latest tools and frameworks for building AI agents and documented the findings in the file 
`findings_3.md`. Here are the key points:

1. **Google Agent Dev Kit (ADK)**
   - Integrates with Gemini and Vertex AI.
   - Modular, component-based architecture.
   - Supports hierarchical agent compositions and custom tool development.
   - Efficient development with under 100 lines of code.
   - [Source](https://www.datacamp.com/blog/best-ai-agents)

2. **LangChain**
   - Connects large language models (LLMs) with external data, APIs, and AI agent tools.
   - Ideal for task automation and multi-step workflows.
   - [Source](https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents)

3. **OpenAI Responses API**
   - Combines the simplicity of the Chat Completions API with the tool use capabilities of the Assistants API.
   - New API primitive for leveraging OpenAI’s built-in tools to build agents.
   - [Source](https://openai.com/index/new-tools-for-building-agents/)

You can find the detailed findings and source URLs in the `findings_3.md` file.

Tokens: ↑ input 65.09K • cache hit 73.94% • ↓ output 3.71K • $ 0.1397

  ✓ Sub-agent 3 complete

✓ All 5 sub-agents complete in 21.4s
✓ Raw findings saved to parallel_findings.md


## Step 5: Synthesize with GPT-5.1

In [6]:
print("=" * 60)
print("SYNTHESIS: Creating comprehensive report (GPT-5.1)...")
print("=" * 60)

# Create synthesis agent
synthesis_agent = Agent(
    llm=synthesis_llm,
    tools=[Tool(name=FileEditorTool.name)],
)

synthesis_conversation = Conversation(agent=synthesis_agent, workspace=cwd)

SYNTHESIS_PROMPT = """
Read `parallel_findings.md` which contains research findings from multiple parallel searches.

Synthesize these into a comprehensive report and write it to `parallel_report.md`:

# [Topic Title]

## Executive Summary
(2-3 paragraph overview)

## Key Findings
(Organize by theme, not by original question order)
(Cite sources inline)

## Analysis & Implications
(What do these findings mean?)

## Conclusion

## References
(All sources with URLs)

Write professionally. Synthesize and connect ideas across the different sub-questions.
"""

synthesis_conversation.send_message(SYNTHESIS_PROMPT)
synthesis_conversation.run()

print("\n✓ Synthesis complete!")

SYNTHESIS: Creating comprehensive report (GPT-5.1)...


[01/23/26 21:11:26] INFO     Created new conversation b7a34578-24d1-411f-834d-355b32c2177f             ]8;id=430155;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py\state.py]8;;\:]8;id=871508;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py#302\302]8;;\
                             State: {'id': UUID('b7a34578-24d1-411f-834d-355b32c2177f'), 'workspace':              
                             {'working_dir': '/Users/rajivshah/Code/openhands-deep-agent', 'kind':                 
                             'LocalWorkspace'}, 'persistence_dir': None, 'max_iterations': 500,                    
                             'stuck_detection': True, 'execution_status':                                          
                             <ConversationExecutionStatus.IDLE: 'idle'>, 'confirmation_policy':                    
                             {'kind': 'NeverConfirm'}, 'security_analyzer': None,                                  
                             'activated_knowledge_skills': [], 'blocked_actions': {},                              
                             'blocked_messages': {}, 'stats': {'usage_to_metrics': {}},                            
                             'secret_registry': {'secret_sources': {}}}                                            
                             Agent: {'llm': {'model': 'openai/gpt-5.1', 'api_key':                                 
                             SecretStr('**********'), 'openrouter_site_url':                                       
                             'https://docs.all-hands.dev/', 'openrouter_app_name': 'OpenHands',                    
                             'num_retries': 5, 'retry_multiplier': 8.0, 'retry_min_wait': 8,                       
                             'retry_max_wait': 64, 'timeout': 300, 'max_message_chars': 30000,                     
                             'temperature': 0.0, 'top_p': 1.0, 'max_input_tokens': 272000,                         
                             'max_output_tokens': 128000, 'stream': False, 'drop_params': True,                    
                             'modify_params': True, 'disable_stop_word': False, 'caching_prompt':                  
                             True, 'log_completions': False, 'log_completions_folder':                             
                             'logs/completions', 'native_tool_calling': True, 'reasoning_effort':                  
                             'high', 'enable_encrypted_reasoning': True, 'prompt_cache_retention':                 
                             '24h', 'extended_thinking_budget': 200000, 'usage_id': 'default',                     
                             'litellm_extra_body': {}}, 'tools': [{'name': 'file_editor', 'params':                
                             {}}], 'mcp_config': {}, 'include_default_tools': ['FinishTool',                       
                             'ThinkTool'], 'system_prompt_filename': 'system_prompt.j2',                           
                             'security_policy_filename': 'security_policy.j2', 'system_prompt_kwargs':             
                             {'llm_security_analyzer': True}, 'kind': 'Agent'}                                     

[01/23/26 21:11:26] INFO     FileEditor initialized with cwd:                                          ]8;id=288367;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/file_editor/editor.py\editor.py]8;;\:]8;id=218626;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/file_editor/editor.py#95\95]8;;\
                             /Users/rajivshah/Code/openhands-deep-agent                                            

[01/23/26 21:11:26] INFO     Loaded 1 tools from spec: ['file_editor']                                  ]8;id=75362;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py\base.py]8;;\:]8;id=266505;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py#281\281]8;;\

System Prompt ─────────────────────────────────────────────────────────────────────────────────────────────────────

System Prompt:
You are OpenHands agent, a helpful AI assistant that can interact with a computer to solve tasks.

<ROLE>
* Your primary role is to assist users by executing commands, modifying code, and solving technical problems 
effectively. You should be thorough, methodical, and prioritize quality over speed.
* If the user asks a question, like "why is X happening", don't try to fix the problem. Just give an answer to the 
question.
</ROLE>

<MEMORY>
* Use `AGENTS.md` under the repository root as your persistent memory for repository-specific knowledge and 
context.
* Add important insights, patterns, and learnings to this file to improve future task performance.
* This repository skill is automatically loaded for every conversation and helps maintain context across sessions.
* For more information about skills, see: https://docs.openhands.dev/overview/skills
</MEMORY>

<EFFICIENCY>
* Each action you take is somewhat expensive. Wherever possible, combine multiple actions into a single action, 
e.g. combine multiple bash commands into one, using sed and grep to edit/view multiple files at once.
* When exploring the codebase, use efficient tools like find, grep, and git commands with appropriate filters to 
minimize unnecessary operations.
</EFFICIENCY>

<FILE_SYSTEM_GUIDELINES>
* When a user provides a file path, do NOT assume it's relative to the current working directory. First explore the
file system to locate the file before working on it.
* If asked to edit a file, edit the file directly, rather than creating a new file with a different filename.
* For global search-and-replace operations, consider using `sed` instead of opening file editors multiple times.
* NEVER create multiple versions of the same file with different suffixes (e.g., file_test.py, file_fix.py, 
file_simple.py). Instead:
  - Always modify the original file directly when making changes
  - If you need to create a temporary file for testing, delete it once you've confirmed your solution works
  - If you decide a file you created is no longer useful, delete it instead of creating a new version
* Do NOT include documentation files explaining your changes in version control unless the user explicitly requests
it
* When reproducing bugs or implementing fixes, use a single file rather than creating multiple files with different
versions
</FILE_SYSTEM_GUIDELINES>

<CODE_QUALITY>
* Write clean, efficient code with minimal comments. Avoid redundancy in comments: Do not repeat information that 
can be easily inferred from the code itself.
* When implementing solutions, focus on making the minimal changes needed to solve the problem.
* Before implementing any changes, first thoroughly understand the codebase through exploration.
* If you are adding a lot of code to a function or file, consider splitting the function or file into smaller 
pieces when appropriate.
* Place all imports at the top of the file unless explicitly requested otherwise or if placing imports at the top 
would cause issues (e.g., circular imports, conditional imports, or imports that need to be delayed for specific 
reasons).
</CODE_QUALITY>

<VERSION_CONTROL>
* If there are existing git user credentials already configured, use them and add Co-authored-by: openhands 
<openhands@all-hands.dev> to any commits messages you make. if a git config doesn't exist use "openhands" as the 
user.name and "openhands@all-hands.dev" as the user.email by default, unless explicitly instructed otherwise.
* Exercise caution with git operations. Do NOT make potentially dangerous changes (e.g., pushing to main, deleting 
repositories) unless explicitly asked to do so.
* When committing changes, use `git status` to see all modified files, and stage all files necessary for the 
commit. Use `git commit -a` whenever possible.
* Do NOT commit files that typically shouldn't go into version control (e.g.

...

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: file_editor
Result:
❌ [An error occurred during execution.]
[An error occurred during execution.]
Invalid `path` parameter: /Users/rajivshah/Code/openhands-deep-agent/parallel_report.md. The path 
/Users/rajivshah/Code/openhands-deep-agent/parallel_report.md does not exist. Please provide a valid path.

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Create synthesized report of AI agent research findings

Thought:
Next I’ll write a synthesized report into a new markdown file.

Reasoning:
Action: FileEditorAction

Arguments:
  command: "create"
  path: "/Users/rajivshah/Code/openhands-deep-agent/parallel_report.md"
  file_text: 
    # Latest Breakthroughs in AI Agents and Autonomous Systems (2024–2025)
    
    ## Executive Summary
    
    Recent literature and market commentary from 2024–2025 point to a decisive shift from single, monolithic 
language-model applications toward composable, autonomous AI agents and multi-agent systems. These agents 
increasingly leverage domain-specific large language models (LLMs), are orchestrated in collaborative swarms, and 
are supported by rapidly maturing tooling such as Google’s Agent Dev Kit, LangChain, and the OpenAI Responses API 
([Medium](https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf); [Capitole 
Consulting](https://www.capitole-consulting.com/blog/turing-to-autonomous-agents-2025-llm-ecosystem); 
[SuperAnnotate](https://www.superannotate.com/blog/multi-agent-llms); 
[DataCamp](https://www.datacamp.com/blog/best-ai-agents); 
[Quashbugs](https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents); 
[OpenAI](https://openai.com/index/new-tools-for-building-agents/)). This evolution promises significant gains in 
capability, efficiency, and scalability across complex workflows.
    
    At the same time, the sources consistently highlight non-technical constraints: limitations in memory and state
management, open questions around governance and autonomous decision-making, and growing concerns regarding trust, 
control, and privacy as agents embed themselves in everyday services ([Medium – Limitations and Challenges of AI 
Agents](https://medium.com/mkinf/limitations-and-challenges-of-ai-agents-67eb49577b10); 
[IBM](https://www.ibm.com/think/insights/ai-agent-governance); [BBC 
R&D](https://www.bbc.co.uk/rd/articles/2025-05-ai-agents-challenges-summary)). Real-world deployments in media, 
business operations, and logistics demonstrate that agentic systems are already generating value, but they also 
illustrate the urgency of responsible design and robust governance frameworks 
([Reddit](https://www.reddit.com/r/AI_Agents/comments/1k68jn7/do_you_guys_know_some_real_world_examples_of/); 
[tkxel](http://tkxel.com/blog/ai-agents-use-cases-2025/); 
[Dialpad](https://www.dialpad.com/blog/agentic-ai-use-cases/)).
    
    ## Key Findings
    
    ### 1. Evolving Architectures: From Single LLMs to Specialized and Collaborative Agents
    
    - **Domain-specific LLMs are central to the next generation of agents.**
      - There is a marked shift toward LLMs tailored to specific industries and use cases, allowing agents to 
reason more effectively over domain language, constraints, and processes 
([Medium](https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf)). This 
specialization supports higher accuracy and more reliable behavior than generic models in complex, regulated, or 
technical domains.
    
    - **The Era of Autonomy is characterized by increasingly capable agentic systems.**
      - The period around 2025–2026 is framed as an Era of Autonomy, in which sophisticated autonomous agents (for 
example, systems like Manus AI) operate with greater independence in complex environments ([Capitole 
Consulting](https://www.capitole-consulting.com/blog/turing-to-autonomous-agents-2025-llm-ecosystem)). These agents
move beyond simple task completion toward continuous, goal-directed behavior.
    
    - **Multi-agent LLM systems enable division of labor and collective problem-solving.**
      - Multi-agent approaches coordinate several LLM-driven agents that collaborate to break down and solve 
com

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: file_editor
Result:
[File /Users/rajivshah/Code/openhands-deep-agent/parallel_report.md edited with 1 changes.]
[begin of edit 1 / 1]
(content before edit)
 1|
(content after edit)
+1|# Latest Breakthroughs in AI Agents and Autonomous Systems (2024–2025)
 2|
+3|## Executive Summary
+4|
+5|Recent literature and market commentary from 2024–2025 point to a decisive shift from single, monolithic 
language-model applications toward composable, autonomous AI agents and multi-agent systems. These agents 
increasingly leverage domain-specific large language models (LLMs), are orchestrated in collaborative swarms, and 
are supported by rapidly maturing tooling such as Google’s Agent Dev Kit, LangChain, and the OpenAI Responses API 
([Medium](https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf); [Capitole 
Consulting](https://www.capitole-consulting.com/blog/turing-to-autonomous-agents-2025-llm-ecosystem); 
[SuperAnnotate](https://www.superannotate.com/blog/multi-agent-llms); 
[DataCamp](https://www.datacamp.com/blog/best-ai-agents); 
[Quashbugs](https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents); 
[OpenAI](https://openai.com/index/new-tools-for-building-agents/)). This evolution promises significant gains in 
capability, efficiency, and scalability across complex workflows.
+6|
+7|At the same time, the sources consistently highlight non-technical constraints: limitations in memory and state 
management, open questions around governance and autonomous decision-making, and growing concerns regarding trust, 
control, and privacy as agents embed themselves in everyday services ([Medium – Limitations and Challenges of AI 
Agents](https://medium.com/mkinf/limitations-and-challenges-of-ai-agents-67eb49577b10); 
[IBM](https://www.ibm.com/think/insights/ai-agent-governance); [BBC 
R&D](https://www.bbc.co.uk/rd/articles/2025-05-ai-agents-challenges-summary)). Real-world deployments in media, 
business operations, and logistics demonstrate that agentic systems are already generating value, but they also 
illustrate the urgency of responsible design and robust governance frameworks 
([Reddit](https://www.reddit.com/r/AI_Agents/comments/1k68jn7/do_you_guys_know_some_real_world_examples_of/); 
[tkxel](http://tkxel.com/blog/ai-agents-use-cases-2025/); 
[Dialpad](https://www.dialpad.com/blog/agentic-ai-use-cases/)).
+8|
+9|## Key Findings
+10|
+11|### 1. Evolving Architectures: From Single LLMs to Specialized and Collaborative Agents
+12|
+13|- **Domain-specific LLMs are central to the next generation of agents.**
+14|  - There is a marked shift toward LLMs tailored to specific industries and use cases, allowing agents to 
reason more effectively over domain language, constraints, and processes 
([Medium](https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf)). This 
specialization supports higher accuracy and more reliable behavior than generic models in complex, regulated, or 
technical domains.
+15|
+16|- **The Era of Autonomy is characterized by increasingly capable agentic systems.**
+17|  - The period around 2025–2026 is framed as an Era of Autonomy, in which sophisticated autonomous agents (for 
example, systems like Manus AI) operate with greater independence in complex environments ([Capitole 
Consulting](https://www.capitole-consulting.com/blog/turing-to-autonomous-agents-2025-llm-ecosystem)). These agents
move beyond simple task completion toward continuous, goal-directed behavior.
+18|
+19|- **Multi-agent LLM systems enable division of labor and collective problem-solving.**
+20|  - Multi-agent approaches coordinate several LLM-driven agents that collaborate to break down and solve 
complex tasks more efficiently ([SuperAnnotate](https://www.superannotate.com/blog/multi-agent-llms)). Instead of a
single, all-purpose model, diff

Message from Agent ────────────────────────────────────────────────────────────────────────────────────────────────

I’ve synthesized the contents of `parallel_findings.md` into a structured, professional report and saved it as:

`parallel_report.md` at the repository root.

It includes:

- A clear topic title aligned with your original research scope  
- A 2-paragraph Executive Summary highlighting trends and risks  
- Key Findings organized by themes (architectures, tooling, use cases, challenges) with inline citations  
- An Analysis & Implications section connecting the trends to governance and adoption considerations  
- A concise Conclusion  
- A complete References section listing all sources with URLs

If you’d like, I can next:  
- Shorten or expand any section for a specific audience, or  
- Add tables, diagrams (described in text), or an executive slide-style summary.

Tokens: ↑ input 30.06K • cache hit 77.93% •  reasoning 2.47K • ↓ output 6.38K • $ 0.0750


✓ Synthesis complete!


## Step 6: View the Report

In [7]:
from IPython.display import Markdown, display

try:
    with open("parallel_report.md", "r") as f:
        display(Markdown(f.read()))
except FileNotFoundError:
    print("parallel_report.md not found - run the cells above first")

# Latest Breakthroughs in AI Agents and Autonomous Systems (2024–2025)

## Executive Summary

Recent literature and market commentary from 2024–2025 point to a decisive shift from single, monolithic language-model applications toward composable, autonomous AI agents and multi-agent systems. These agents increasingly leverage domain-specific large language models (LLMs), are orchestrated in collaborative swarms, and are supported by rapidly maturing tooling such as Google’s Agent Dev Kit, LangChain, and the OpenAI Responses API ([Medium](https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf); [Capitole Consulting](https://www.capitole-consulting.com/blog/turing-to-autonomous-agents-2025-llm-ecosystem); [SuperAnnotate](https://www.superannotate.com/blog/multi-agent-llms); [DataCamp](https://www.datacamp.com/blog/best-ai-agents); [Quashbugs](https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents); [OpenAI](https://openai.com/index/new-tools-for-building-agents/)). This evolution promises significant gains in capability, efficiency, and scalability across complex workflows.

At the same time, the sources consistently highlight non-technical constraints: limitations in memory and state management, open questions around governance and autonomous decision-making, and growing concerns regarding trust, control, and privacy as agents embed themselves in everyday services ([Medium – Limitations and Challenges of AI Agents](https://medium.com/mkinf/limitations-and-challenges-of-ai-agents-67eb49577b10); [IBM](https://www.ibm.com/think/insights/ai-agent-governance); [BBC R&D](https://www.bbc.co.uk/rd/articles/2025-05-ai-agents-challenges-summary)). Real-world deployments in media, business operations, and logistics demonstrate that agentic systems are already generating value, but they also illustrate the urgency of responsible design and robust governance frameworks ([Reddit](https://www.reddit.com/r/AI_Agents/comments/1k68jn7/do_you_guys_know_some_real_world_examples_of/); [tkxel](http://tkxel.com/blog/ai-agents-use-cases-2025/); [Dialpad](https://www.dialpad.com/blog/agentic-ai-use-cases/)).

## Key Findings

### 1. Evolving Architectures: From Single LLMs to Specialized and Collaborative Agents

- **Domain-specific LLMs are central to the next generation of agents.**
  - There is a marked shift toward LLMs tailored to specific industries and use cases, allowing agents to reason more effectively over domain language, constraints, and processes ([Medium](https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf)). This specialization supports higher accuracy and more reliable behavior than generic models in complex, regulated, or technical domains.

- **The Era of Autonomy is characterized by increasingly capable agentic systems.**
  - The period around 2025–2026 is framed as an Era of Autonomy, in which sophisticated autonomous agents (for example, systems like Manus AI) operate with greater independence in complex environments ([Capitole Consulting](https://www.capitole-consulting.com/blog/turing-to-autonomous-agents-2025-llm-ecosystem)). These agents move beyond simple task completion toward continuous, goal-directed behavior.

- **Multi-agent LLM systems enable division of labor and collective problem-solving.**
  - Multi-agent approaches coordinate several LLM-driven agents that collaborate to break down and solve complex tasks more efficiently ([SuperAnnotate](https://www.superannotate.com/blog/multi-agent-llms)). Instead of a single, all-purpose model, different agents can specialize in subtasks such as planning, data retrieval, execution, and verification.
  - This contrasts with a single “godlike” agent, offering improved trade-offs between cost, speed, and quality by distributing work across focused agents ([LinkedIn](https://www.linkedin.com/pulse/multi-agent-systems-future-ai-collaboration-saigon-technology-vgboc)).

- **Collaboration, communication, and specialization are becoming standard design patterns.**
  - Multi-agent systems now routinely incorporate explicit communication protocols, shared planning, and role specialization so that agents can coordinate decisions and actions across a workflow ([Lunavi](https://www.lunavi.com/blog/how-multi-agent-ai-systems-collaborate-to-solve-complex-problems)).
  - These systems are deliberately designed to be scalable and adaptable, forming interconnected networks of agents that handle complex workflows more effectively than isolated models ([ML6](https://www.ml6.eu/en/blog/multi-agent-ai-systems-where-they-shine-and-how-they-work-together)).

### 2. Tooling, Frameworks, and Platform Primitives for Agent Development

- **Google Agent Dev Kit (ADK) lowers the barrier to production agents.**
  - ADK integrates natively with Gemini and Vertex AI, providing a modular, component-based architecture with support for hierarchical agent compositions ([DataCamp](https://www.datacamp.com/blog/best-ai-agents)).
  - It enables relatively complex agents to be built in under 100 lines of code and supports custom tool development, emphasizing rapid iteration and extensibility ([DataCamp](https://www.datacamp.com/blog/best-ai-agents)).

- **LangChain has become a de facto standard for orchestration and integration.**
  - LangChain connects LLMs to external data sources, APIs, and tools, making it particularly well-suited for task automation and multi-step workflows ([Quashbugs](https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents)).
  - Its abstractions around chains, tools, and agents facilitate the construction of complex pipelines that can integrate perception, reasoning, and action.

- **The OpenAI Responses API introduces a new primitive for agent capabilities.**
  - The Responses API combines the simplicity of the Chat Completions API with the tool-use functionality previously associated with the Assistants API, creating a unified primitive for building agents ([OpenAI](https://openai.com/index/new-tools-for-building-agents/)).
  - By exposing built-in tools through a single interface, it simplifies agent design and reduces the need for bespoke tooling layers, encouraging a more standardized approach to agent construction ([OpenAI](https://openai.com/index/new-tools-for-building-agents/)).

### 3. Real-World Adoption and Emerging Use Cases

- **Media and content: AI news anchors as persistent autonomous presenters.**
  - A47’s deployment of 47 AI news anchors that generate and deliver content 24/7 illustrates how agentic systems are being used to automate continuous media production and presentation ([Reddit](https://www.reddit.com/r/AI_Agents/comments/1k68jn7/do_you_guys_know_some_real_world_examples_of/)).

- **Business operations: Automation and personalization at scale.**
  - Organizations are adopting AI agents to automate business processes and create personalized experiences for users, with expectations that these applications will significantly shape multiple industries by 2025 ([tkxel](http://tkxel.com/blog/ai-agents-use-cases-2025/)).
  - These agents are positioned as always-on digital workers that can handle repetitive tasks, respond to customers, and dynamically adapt experiences.

- **Supply chain and logistics: Autonomous operational coordination.**
  - In inventory management and logistics, AI agents detect low stock in high-demand locations, shift inventory from other warehouses, notify vendors, and update logistics systems, demonstrating end-to-end operational orchestration ([Dialpad](https://www.dialpad.com/blog/agentic-ai-use-cases/)).

### 4. Cross-Cutting Challenges and Limitations

- **Memory and state management remain core technical bottlenecks.**
  - Current agents struggle with limitations around memory capacity and state management, which can impair their ability to retain context, learn over time, and make consistent decisions across long-running tasks ([Medium – Limitations and Challenges of AI Agents](https://medium.com/mkinf/limitations-and-challenges-of-ai-agents-67eb49577b10)).

- **Governance and autonomous decision-making raise accountability concerns.**
  - As agents gain more autonomy, questions emerge about who is accountable for their decisions, how to constrain or override them, and how to align their actions with organizational and societal expectations ([IBM](https://www.ibm.com/think/insights/ai-agent-governance)).
  - Unlike traditional deterministic software, agents can make context-dependent decisions in novel situations, complicating auditability and oversight ([IBM](https://www.ibm.com/think/insights/ai-agent-governance)).

- **Trust, control, and privacy are critical adoption barriers.**
  - The growing integration of AI agents into everyday life raises the stakes for secure operation, user control, and privacy protection ([BBC R&D](https://www.bbc.co.uk/rd/articles/2025-05-ai-agents-challenges-summary)).
  - Mistrust in opaque decision-making, potential misuse of data, and lack of clear user controls can slow or prevent deployment even when technical capabilities are strong.

## Analysis & Implications

Taken together, these findings portray a maturing ecosystem in which AI agents are moving from experimental prototypes to production-grade systems embedded across business and consumer contexts. Architecturally, the field is converging on patterns that emphasize specialization, collaboration, and modularity: domain-specific models for depth, multi-agent orchestration for breadth, and standardized tool interfaces for robust integration ([Medium](https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf); [SuperAnnotate](https://www.superannotate.com/blog/multi-agent-llms); [Quashbugs](https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents)). This suggests that competitive advantage will increasingly derive from how effectively organizations design and orchestrate constellations of agents, rather than from access to a single, monolithic model.

The rapid emergence of frameworks and platform primitives such as ADK, LangChain, and the OpenAI Responses API further lowers the barrier to entry, enabling smaller teams to assemble sophisticated systems leveraging cloud-native tooling ([DataCamp](https://www.datacamp.com/blog/best-ai-agents); [Quashbugs](https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents); [OpenAI](https://openai.com/index/new-tools-for-building-agents/)). As a result, we can expect a broad democratization of agent development, with many more domain experts—rather than only AI specialists—participating in the construction of agents tailored to their workflows. This democratization, combined with increasing autonomy, will likely accelerate deployment across sectors such as media, customer experience, and operations, as evidenced by early use cases like AI news anchors and autonomous inventory management ([Reddit](https://www.reddit.com/r/AI_Agents/comments/1k68jn7/do_you_guys_know_some_real_world_examples_of/); [Dialpad](https://www.dialpad.com/blog/agentic-ai-use-cases/)).

However, the technical and governance challenges highlighted by the sources indicate that raw capability alone will not determine success. Memory and state limitations constrain agents’ ability to operate as persistent, continuously learning entities, which is essential for long-lived workflows and enterprise knowledge management ([Medium – Limitations and Challenges of AI Agents](https://medium.com/mkinf/limitations-and-challenges-of-ai-agents-67eb49577b10)). Governance and accountability concerns require new organizational structures, policies, and tooling for monitoring, auditing, and controlling autonomous decisions ([IBM](https://www.ibm.com/think/insights/ai-agent-governance)). Meanwhile, unresolved issues of trust, control, and privacy may generate user and regulatory resistance if not addressed proactively ([BBC R&D](https://www.bbc.co.uk/rd/articles/2025-05-ai-agents-challenges-summary)).

The implication is that organizations cannot treat agent adoption purely as a technical upgrade. They must co-design architectures, processes, and governance frameworks. Multi-agent designs, for example, can be used not only for performance but also for safety—such as assigning distinct agents for planning, execution, and oversight, or incorporating review agents that critique and constrain the actions of others ([SuperAnnotate](https://www.superannotate.com/blog/multi-agent-llms); [Lunavi](https://www.lunavi.com/blog/how-multi-agent-ai-systems-collaborate-to-solve-complex-problems)). Similarly, domain-specific models can embed regulatory and policy constraints within the model’s training data and prompts, reducing the risk of non-compliant behavior in sensitive sectors.

## Conclusion

The 2024–2025 period marks a pivotal phase in the evolution of AI agents and autonomous systems. Advances in domain-specific LLMs, multi-agent collaboration, and standardized tooling are enabling powerful, scalable, and increasingly autonomous agents that can operate across complex workflows and industries. Early real-world deployments in media, business automation, and logistics demonstrate tangible value and signal the trajectory toward broader adoption.

At the same time, these advances surface pressing challenges around memory, statefulness, governance, accountability, and user trust. The most successful organizations will be those that pair technical innovation with robust governance, clear oversight mechanisms, and privacy-by-design principles. In this landscape, agentic AI should be viewed not as a single technology, but as a socio-technical system requiring integrated attention to architecture, operations, and ethics.

## References

- Medium – 15 Artificial Intelligence LLM Trends in 2024: https://medium.com/data-bistrot/15-artificial-intelligence-llm-trends-in-2024-618a058c9fdf
- Capitole Consulting – Turing to Autonomous Agents 2025 LLM Ecosystem: https://www.capitole-consulting.com/blog/turing-to-autonomous-agents-2025-llm-ecosystem
- SuperAnnotate – Multi-Agent LLMs: https://www.superannotate.com/blog/multi-agent-llms
- LinkedIn – Multi-Agent Systems and the Future of AI Collaboration: https://www.linkedin.com/pulse/multi-agent-systems-future-ai-collaboration-saigon-technology-vgboc
- Lunavi – How Multi-Agent AI Systems Collaborate to Solve Complex Problems: https://www.lunavi.com/blog/how-multi-agent-ai-systems-collaborate-to-solve-complex-problems
- ML6 – Multi-Agent AI Systems: Where They Shine and How They Work Together: https://www.ml6.eu/en/blog/multi-agent-ai-systems-where-they-shine-and-how-they-work-together
- DataCamp – Best AI Agents: https://www.datacamp.com/blog/best-ai-agents
- Quashbugs – Top Tools and Frameworks for Building AI Agents: https://quashbugs.com/blog/top-tools-frameworks-building-ai-agents
- OpenAI – New Tools for Building Agents: https://openai.com/index/new-tools-for-building-agents/
- Medium – Limitations and Challenges of AI Agents: https://medium.com/mkinf/limitations-and-challenges-of-ai-agents-67eb49577b10
- IBM – AI Agent Governance: Big Challenges, Big Opportunities: https://www.ibm.com/think/insights/ai-agent-governance
- BBC R&D – AI Agents: Exploring the Potential and the Problems: https://www.bbc.co.uk/rd/articles/2025-05-ai-agents-challenges-summary
- Reddit – Real-World Examples of AI Agents: https://www.reddit.com/r/AI_Agents/comments/1k68jn7/do_you_guys_know_some_real_world_examples_of/
- tkxel – AI Agents Use Cases 2025: http://tkxel.com/blog/ai-agents-use-cases-2025/
- Dialpad – Agentic AI Use Cases: https://www.dialpad.com/blog/agentic-ai-use-cases/

## Summary: Sequential vs Parallel

| Approach | Notebook 01 | This Notebook |
|----------|-------------|---------------|
| **Execution** | Sequential | Parallel (ThreadPoolExecutor) |
| **Sub-agents** | 1 agent, multiple turns | N agents, isolated context |
| **Speed** | ~N × search_time | ~1 × search_time |
| **Context** | Shared (can grow large) | Isolated (stays small) |
| **Error handling** | One failure stops all | Failures isolated |

**When to use parallel:**
- Many independent sub-questions
- Speed is important
- Sub-questions don't depend on each other

**When to use sequential (Notebook 01):**
- Sub-questions build on each other
- Need to refine queries based on earlier results
- Simpler debugging